# Load Test
This test is supposed to run alongside a LOVE deployment with the following components:
- LOVE-manager
- LOVE-commander
- LOVE-events-producer

The test instantiates a series of fake clients that connect to the LOVE manager and subscribe to events, and a series of emitters that send commands to a Test-CSC which trigger events.
The test calculates events reception latencies on the client side

## Test parameters
Configure number of clients, emitters and other parameters below:

In [2]:
manager_location = 'love-manager:8000'
# Change to line below when running from live environment
# manager_location = 'love-manager-mount:8000'

N_CLIENTS = 5 # Number of clients to simulate
N_EMITTERS = 5 # Number of emitters that will send messages
N_EMITTER_COMMANDS = 50 # Number of commands that each emitter will run

---
## Test execution
### Test setup
Classes necessary for the test

In [3]:
# Most of this was adapted from https://github.com/lsst-ts/ts_salobj/blob/develop/tests/test_csc.py
import asyncio
import json
from lsst.ts import salobj
import logging

STD_TIMEOUT = 15  # timeout for command ack
LONG_TIMEOUT = 30  # timeout for CSCs to start
EVENT_DELAY = 0.1  # time for events to be output as a result of a command
NODATA_TIMEOUT = 0.1  # timeout for when we expect no new data
SHOW_LOG_MESSAGES = False


class FailedCallbackCsc(salobj.TestCsc):
    """A CSC whose do_wait command raises a RuntimeError and whose do_fault command includes a report"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.exc_msg = "do_wait raised an exception on purpose on Test-{}".format(kwargs["index"])
        self.index = kwargs["index"]
        self.log.warning = lambda x: {}
        self.log.info = lambda x: {}
        self.log.error = lambda x: {}
        self.log.debug = lambda x: {}

    async def do_wait(self, data):
        raise RuntimeError(self.exc_msg)

    async def do_fault(self, data):
        """Execute the fault command with a report, code and traceback.
        Change the summary state to State.FAULT
        """
        self.log.warning("Executing fault on Test-{}".format(self.index))
        code = 52
        report = "Report for error code for Test-{}".format(self.index,)
        traceback = "Traceback for error code for Test-{}".format(self.index,)
        self.fault(code=code, report=report, traceback=traceback)


class LogMessagesMock():
    """Triggers logMessages and errorCode events in a TestCSC """

    def __init__(self, salindex, *args, **kwargs):
        self.csc = FailedCallbackCsc(index=salindex, *args, **kwargs)
        d = salobj.Domain()
        self.r = salobj.Remote(d, 'Test', salindex)
        self.salindex = salindex

    async def set_log_level(self):
        await self.r.cmd_setLogLevel.set_start(level=logging.DEBUG, timeout=STD_TIMEOUT)

    def log_info_message(self):
        info_message = "test info message for Test-{}".format(self.salindex,)
        self.csc.log.info(info_message)

    def log_warn_message(self):
        warn_message = "test warn message for Test-{}".format(self.salindex,)
        self.csc.log.warning(warn_message)

    async def log_error_message(self):
        with salobj.assertRaisesAckError():
            await self.r.cmd_wait.set_start(duration=5, timeout=STD_TIMEOUT)

    async def printmessage(self):
        msg = await self.r.evt_logMessage.next(flush=True)
        print('\n TestCSC', self.salindex, ' | msg:', msg.message, '\nlvl:', msg.level, '\ntrace:', msg.traceback)


async def launch(salindex, debug=False):
    mock = LogMessagesMock(salindex, initial_state=salobj.State.ENABLED)
    asyncio.ensure_future(mock.csc.done_task)
    await mock.set_log_level()
    logmessages = [
        mock.log_info_message,
        mock.log_warn_message,
        mock.log_error_message
    ]

    counter = 1

    while True:
        for index, message in enumerate(logmessages):
            if index == 2:
                await message()
            else:
                message()
            if debug:
                while True:
                    try:
                        await mock.printmessage()
                    except asyncio.TimeoutError:
                        break
            counter += 1
            await asyncio.sleep(5)

def create_cscs(number):
    awaitables = []
    loop = asyncio.get_event_loop()
    for salindex in range(1, number+1):
        awaitables.append(loop.create_task(launch(salindex, True)))
    return awaitables

In [4]:
import aiohttp
import json
import asyncio
import requests
from astropy.time import Time

class clientInstance:
    
    def __init__(self):
        self.latencies = []
        self.websocket_url = ''
        self.received_messages = 0
    
    def request_token(self):
        url = f'http://{manager_location}/manager/api/get-token/'
        data = {
            'username': 'test',
            'password': 'test',
        }
        resp = requests.post(url, data = data)
        token = resp.json()['token']
        print(token)
        self.websocket_url = f'ws://{manager_location}/manager/ws/subscription?token={token}'
    
    async def handle_message_reception(self):
        """Handles the reception of messages"""
        if self.websocket:
            async for message in self.websocket:
                if message.type == aiohttp.WSMsgType.TEXT:
                    msg = json.loads(message.data)
                    if 'category' not in msg or msg['category'] != 'event' or "option" in msg and msg["option"] == "subscribe":
                        continue
                    self.received_messages = self.received_messages + 1
                    cur_time = Time.now().tai.datetime.timestamp()
                    snd_time = msg['data'][0]['data']['summaryState'][0]['private_sndStamp']['value']
                    self.latencies.append(cur_time - snd_time)
    
    async def subscribe_to_salindex(self, salindex):
        subscribe_msg = {
            'option': 'subscribe',
            'category': 'event',
            'csc': 'Test',
            'salindex': f'{salindex}',
            'stream': 'summaryState'
        }
        await self.websocket.send_str(json.dumps(subscribe_msg))
    
    async def start_ws_client(self, number):
        async with aiohttp.ClientSession() as session:
            self.websocket = await session.ws_connect(self.websocket_url)
            # async with websockets.connect(websocket_url) as websocket:
            print('started client')
            for i in range(1, N_EMITTERS+1):
                await self.subscribe_to_salindex(i)
                
            await self.handle_message_reception()


In [5]:
from lsst.ts import salobj
import asyncio

class Emitter:
    
    def __init__(self, *args, **kwargs):
        self.salindex = kwargs["salindex"]
        self.emitted_commands = 0
        
    async def create_emitter_task(self):
        d = salobj.Domain()
        r = salobj.Remote(d, 'Test', self.salindex)

        cmds = [
            r.cmd_enable,
            r.cmd_fault,
            r.cmd_standby,
            r.cmd_start,
        ]
        for i in range(N_EMITTER_COMMANDS):
            for command in cmds:
                try:
                    await command.start()
                    self.emitted_commands = self.emitted_commands + 1
                except Exception as e:
                    print('Test CSC error:', e)
                await asyncio.sleep(0.1)
            
def create_emitters(number):
    emitters = []
    for salindex in range(1, number+1):
        emitters.append(Emitter(salindex=salindex))
    return emitters

### Run test
#### Create and launch clients
Clients are created and run, once they start they connect to the LOVE-manager

In [6]:
# Create clients and listen to ws messages
loop = asyncio.get_event_loop()
clients = [clientInstance() for i in range(N_CLIENTS)]
[c.request_token() for c in clients]
client_tasks = [loop.create_task(c.start_ws_client(N_EMITTERS)) for c in clients]

94b554fefd91092fb563d6a99ad1af6bdfd05ee5
05db08e40a58251063f6dd9b37f7c57313d70c07
d28411291dad706747e2fabce5f2c7a6b63c8394
b372b56e8e24074e3eeb82c2adba46acaf8f5b1c
0a4381b307c08fd32deb84ae7a09e5cd124a1b24


#### Create and launch emitters
We wait for 5 seconds, then emitters are created and run.
Once they run they start sending commands (which trigger the events)

In [7]:
# Create emitters and CSCs
import time
time.sleep(5)
loop = asyncio.get_event_loop()
csc_tasks = create_cscs(N_EMITTERS)
emitters = create_emitters(N_EMITTERS)
emitter_tasks = [loop.create_task(e.create_emitter_task()) for e in emitters]

backend_tasks = [*csc_tasks, *emitter_tasks]
# [t.cancel() for t in tasks]
# await asyncio.gather(*tasks)

#### Wait for emitters to end
Wait for emitters to finish their task. This will take a while...

In [8]:
# Wait for emitters
await asyncio.gather(*emitter_tasks)

Test CSC error: msg='Command failed', ackcmd=(ackcmd private_seqNum=433765332, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: enable not allowed in state <State.ENABLED: 2>')
Test CSC error: msg='Command failed', ackcmd=(ackcmd private_seqNum=355790519, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: enable not allowed in state <State.ENABLED: 2>')
Test CSC error: msg='Command failed', ackcmd=(ackcmd private_seqNum=450908377, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: enable not allowed in state <State.ENABLED: 2>')
Test CSC error: msg='Command failed', ackcmd=(ackcmd private_seqNum=361867316, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: enable not allowed in state <State.ENABLED: 2>')
Test CSC error: msg='Command failed', ackcmd=(ackcmd private_seqNum=427514389, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: enable not allowed in state <State.ENABLED: 2>')
started client
started client
started client
started client


[None, None, None, None, None]

---
# Report
Here are the test results

## Summary of received messages

In [9]:
client_msgs = [c.received_messages for c in clients]
emitter_msgs = [e.emitted_commands for e in emitters]
print(f'Sent {sum(emitter_msgs)} messages, received {sum(client_msgs)} messages ({sum(client_msgs)/len(clients)} per client)')

Sent 995 messages, received 0 messages (0.0 per client)


## Latency
### Average and median latency
Here are some latency statistics

In [10]:
import statistics

latencies = [[l*1000 for l in c.latencies] for c in clients]
avg_latencies = [statistics.mean(l) for l in latencies]
median_latencies = [statistics.median(l) for l in latencies]
avg_latency = statistics.mean(avg_latencies)
median_latency = statistics.median(median_latencies)

print(f'Average message latency: {round(avg_latency)}ms')
print(f'Median message latency: {round(median_latency)}ms')


StatisticsError: mean requires at least one data point

### Latencies series
The following chart represents the latencies of each client sorted by reception. That is, the leftmost value was the first message to be received

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 8]
for l in latencies:
    plt.plot(range(len(l)), l)

In [ ]:
import numpy as np
sum(np.array(latencies[0])-np.array(latencies[4]))

---
## Test cleanup
Wait for all tasks to end

In [ ]:
[t.cancel() for t in [*backend_tasks, *client_tasks]]
#[t.cancel() for t in [*client_tasks]]
for task in asyncio.Task.all_tasks():
    task.cancel()